In [1]:
from tm2py.acceptance import Acceptance
import numpy as np
import pandas as pd
import geopandas as gpd
import os

In [2]:
scenario_config_file = "../examples/scenario_config.toml"
model_config_file = "../examples/model_config.toml"
observed_config_file = "../examples/observed_data.toml"

In [3]:
a = Acceptance(scenario_config_file, model_config_file, observed_config_file)

In [4]:
a.make_acceptance(make_transit=False, make_roadway=False, make_other=True)

In [ ]:
# start here, put the above code in a method
# then do the comparison and stick in other
# get tableau going

In [ ]:
self = a
a_df = self._make_home_work_flow_comparisons()
b_gdf = self._make_zero_vehicle_household_comparisons()

gdf = gpd.GeoDataFrame(pd.concat([a_df, b_gdf]), geometry="geometry")

In [ ]:
a_df.head()

In [ ]:
b_gdf.head()

In [ ]:
b_gdf.dtypes

In [ ]:
self = a
a_df = pd.merge(
    self.simulated_zero_vehicle_hhs_df,
    self.simulated_maz_data_df[["MAZ_ORIGINAL", "MAZSEQ"]],
    left_on="maz",
    right_on="MAZSEQ",
    how="left",
).drop(columns=["maz", "MAZSEQ"]).rename(columns={"MAZ_ORIGINAL": "maz"})

a_df = pd.merge(
    a_df,
    self.census_2010_to_maz_crosswalk_df,
    how="left",
    on="maz",
)

a_df["product"] = a_df["simulated_zero_vehicle_share"] * a_df["maz_share"]

b_df = a_df.groupby("blockgroup").agg({"product": "sum", "simulated_households": "sum"}).reset_index().rename(columns={"product": "simulated_zero_vehicle_share"})
b_df["tract"] = b_df["blockgroup"].astype("str").str.slice(stop=-1)
b_df["product"]= b_df["simulated_zero_vehicle_share"] * b_df["simulated_households"]

c_df = b_df.groupby("tract").agg({"product": "sum", "simulated_households": "sum"}).reset_index()
c_df["simulated_zero_vehicle_share"] = c_df["product"] / c_df["simulated_households"]

# prepare the observed data
join_df = self.census_2017_zero_vehicle_hhs_df.copy()
join_df["tract"] = join_df["geoid"].str.replace("1400000US0", "")
join_df = join_df[["tract", "observed_zero_vehicle_household_share"]].drop_duplicates()

df = pd.merge(
    c_df[["tract", "simulated_zero_vehicle_share"]],
    join_df,
    how="left",
    on="tract",
)

df = pd.merge(
    df, 
    self.census_tract_centroids_gdf, 
    how="left", 
    on="tract"
)

In [ ]:
df.head()

In [ ]:
gdf = self.census_tract_centroids_gdf
gdf.head()

In [ ]:
k_df = pd.DataFrame(gdf[(gdf["tract"] == "06001400200")].copy()).drop(columns=["geometry"])
k_df.head()

In [ ]:
k_df.tract.str.slice(start=1).head()